In [19]:
import copy
import glob
import json
import os
import random
import re
from collections import Counter
from random import shuffle

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [20]:
random.seed(0)

In [21]:
root_dir = "/home/kirill/Documents/1.Projects/" \
           "class-ranking/intellij-community/" \
           "project-processing-results/processing/" \
           "java/classes/processing/0.0.1/"

cnt = 0
individualUsages = {}  # {className -> {className -> count}}
graph = {}  #{className -> [parentNames]}

def good(usage) -> bool:
    # {'TYPE', 'THIS_EXPRESSION', 'EXTENDS_LIST', 'IMPLEMENTS_LIST', 'METHOD_REF_EXPRESSION', 'EXPRESSION_LIST', 'NEW_EXPRESSION', 'CONDITIONAL_EXPRESSION', 'FIELD', 'ANNOTATION', 'THROWS_LIST', 'SUPER_EXPRESSION', 'JAVA_CODE_REFERENCE', 'REFERENCE_EXPRESSION', 'DOC_REFERENCE_HOLDER'}
    if usage["features"]["referenceType"] == "ANNOTATION":
        return False
    if usage["features"]["referenceType"] == "DOC_REFERENCE_HOLDER":
        return False
    return True

def addToUsages(data, individualUsages):
    enclosingName = data["keyInfo"]["name"]
    c = Counter()
    for usage in data['usages']:
        if good(usage):
            c.update([usage['name']])
    individualUsages[enclosingName] = c

def addToGraph(data, graph):
    enclosingName = data["keyInfo"]["name"]
    extends = data["keyInfo"]["additionalInfo"]["enclosingClassExtendsList"]
    impls = data["keyInfo"]["additionalInfo"]["enclosingClassImplementsList"]
    if enclosingName not in graph:
        graph[enclosingName] = set()
    for par in (extends + impls):
        if par not in graph:
            graph[par] = set()
        graph[par].update({enclosingName})


print(f'load files, build a graph, count indevidual usages ')
for filename in glob.glob(root_dir + '**/*.json', recursive=True):
    #print(f'name {filename}')
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        data = json.load(f)
        addToUsages(data, individualUsages)
        addToGraph(data, graph)
        cnt += 1
        if cnt % 10**4 == 0:
            print(f'{cnt} files loaded, for current file: direct_usages = {len(individualUsages[data["keyInfo"]["name"]])} | direct_children = {len(graph[data["keyInfo"]["name"]])} ')
print(f'{cnt} files processed ')

load files, build a graph, count indevidual usages 
10000 files loaded, for current file: direct_usages = 6 | direct_children = 0 
20000 files loaded, for current file: direct_usages = 10 | direct_children = 0 
30000 files loaded, for current file: direct_usages = 2 | direct_children = 0 
40000 files loaded, for current file: direct_usages = 5 | direct_children = 0 
50000 files loaded, for current file: direct_usages = 2 | direct_children = 0 
53265 files processed 


In [22]:
familyUsages = {} # className -> usages in all children (className -> number)
family = {} # className -> self and all children set(className)
parent = {} # className -> className
def dfs(v):
    cur_usages = copy.deepcopy(individualUsages[v])
    cur_family_set = {v}
    for u in graph[v]:
        if u not in family:
            parent[u] = v
            dfs(u)

        u_usages, u_child_set = familyUsages[u], family[u]
        cur_usages.update(u_usages)
        cur_family_set.update(u_child_set)
    familyUsages[v], family[v] = cur_usages, cur_family_set


v_count = 0
for v in individualUsages:
    if v not in familyUsages:
        dfs(v)
    v_count += 1
    if v_count % 10**4 == 0:
        print(f'{v_count} vertexes evalueted | in {v} used {len(familyUsages[v])} classes, {len(family[v])} children')

10000 vertexes evalueted | in com.intellij.openapi.externalSystem.model.execution.ExternalSystemTaskExecutionSettings used 6 classes, 1 children
20000 vertexes evalueted | in com.intellij.codeInspection.SimplifyCollectorInspection used 10 classes, 1 children
30000 vertexes evalueted | in com.intellij.refactoring.changeSignature.ChangeSignatureParameterUsageInfo used 2 classes, 1 children
40000 vertexes evalueted | in com.intellij.structuralsearch.impl.matcher.MatchResultImpl used 5 classes, 1 children
50000 vertexes evalueted | in com.intellij.execution.actions.ChooseDebugConfigurationPopupAction used 2 classes, 1 children


In [23]:
ROOT_CLASS = "com.intellij.openapi.actionSystem.AnAction"
#print(familyUsages[ROOT_CLASS].most_common())

In [24]:
train_test_classes = family[ROOT_CLASS]
train_list, test_list = train_test_split(list(train_test_classes), train_size=0.7, random_state=0)
test_list, valid_list = train_test_split(test_list, train_size=0.65, random_state=0)
trainClasses = set(train_list)
testClasses = set(test_list)
validClasses = set(valid_list)

In [25]:
train_usages = set()
for c in trainClasses:
    train_usages.update(individualUsages[c].keys())

In [26]:
firstCharToNameList = {}
for k, v in familyUsages[ROOT_CLASS].most_common():
    if k not in train_usages:
        continue
    first_char = k.rpartition('.')[2][0]
    if first_char not in firstCharToNameList:
        firstCharToNameList[first_char] = []
    firstCharToNameList[first_char].append(k)

with open('FirstCharToNameList.txt', 'w') as convert_file:
     convert_file.write(json.dumps(firstCharToNameList))

# for k in firstCharToNameList:
#     print(f'{k} {len(firstCharToNameList[k])}')

In [27]:
firstCharToNameListSampling = {}
for k, v in familyUsages[ROOT_CLASS].most_common():
    if k not in train_usages:
        continue
    first_char = k.rpartition('.')[2][0]
    if first_char not in firstCharToNameListSampling:
        firstCharToNameListSampling[first_char] = []
    firstCharToNameListSampling[first_char] += [k] * v

for k in firstCharToNameListSampling:
    shuffle(firstCharToNameListSampling[k])

with open('FirstCharToNameListSampling.txt', 'w') as convert_file:
     convert_file.write(json.dumps(firstCharToNameListSampling))

In [28]:
def listToStr(df, col):
    tmp = df[col].isnull(), col
    df.loc[tmp] = df.loc[tmp].apply(lambda x: [])
    df[col] = df[col].apply(lambda x: " ".join(x) )

In [29]:


cnt = 0
cntu = 0
list_of_dfs_train = []
list_of_dfs_test = []
list_of_dfs_valid = []
group_id = 0
for filename in glob.glob(root_dir + '**/*.json', recursive=True):
    enclosingClassName = filename.rpartition('/')[0].rpartition('/')[2]
    if enclosingClassName not in train_test_classes:
            continue
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        data = json.load(f)
        data_usages = []
        for usage in data["usages"]:
            if not good(usage):
                continue
            usage['correct'] = 1
            usage['firstChar'] = usage['name'].rpartition('.')[2][0]
            usage['group'] = group_id
            group_id += 1
            data_usages.append(usage)

            if enclosingClassName in trainClasses:
                negatives = 8
            else:
                negatives = 25

            for i in range(negatives):
                usage_negative = copy.deepcopy(usage)
                while True:
                    new_name = random.choice(firstCharToNameListSampling[usage['firstChar']]) #?
                    if usage['name'] != new_name:
                        usage_negative['name'] = new_name
                        usage_negative['correct'] = 0
                        break
                data_usages.append(usage_negative)

        df = pd.json_normalize(data_usages)
        df['features.enclosingClassName'] = enclosingClassName

        df = df.drop(columns=['filePath', 'textOffset', 'features.lineInFile'])
        if enclosingClassName in trainClasses:
            list_of_dfs_train.append(df)
        if enclosingClassName in testClasses:
            list_of_dfs_test.append(df)
        if enclosingClassName in validClasses:
            list_of_dfs_valid.append(df)
        cntu += len(data_usages)
        cnt += 1

df_train_raw = pd.concat(list_of_dfs_train, axis=0, ignore_index=True)
df_test_raw = pd.concat(list_of_dfs_test, axis=0, ignore_index=True)
df_valid_raw = pd.concat(list_of_dfs_valid, axis=0, ignore_index=True)

listToStr(df_train_raw, 'features.variablesTypesInScope')
listToStr(df_test_raw, 'features.variablesTypesInScope')
listToStr(df_valid_raw, 'features.variablesTypesInScope')
df_train_raw.fillna('', inplace=True)
df_test_raw.fillna('', inplace=True)
df_valid_raw.fillna('', inplace=True)

print(f'{cnt} files loaded, {cntu} usages')

print(f'test shape {df_test_raw.shape}')
print(f'train shape {df_train_raw.shape}')
print(f'valid shape {df_valid_raw.shape}')
df_train_raw.dtypes

# will we know referenceType irl ?

2810 files loaded, 604175 usages
test shape (218764, 12)
train shape (271557, 12)
valid shape (113854, 12)


name                              object
correct                            int64
firstChar                         object
group                              int64
features.referenceType            object
features.scopeKind                object
features.enclosingScopeName       object
features.variablesTypesInScope    object
features.insideStatement_0        object
features.enclosingClassName       object
features.insideStatement_1        object
features.insideStatement_2        object
dtype: object

In [30]:
def splitCamelCase(name: str):
    res = []
    for rname in name.split(' '):
        rname = rname.strip('.')
        if '.' in rname:
            rname = rname.rpartition('.')[2]
        res += re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', rname)).split()
        #res += [rname]
    return res

In [31]:
def str_to_OHE_feature(df_train, df_test, df_valid, col, pref, max_features=85, replace_columns=True):
    vectorizer_train = CountVectorizer(tokenizer=splitCamelCase, lowercase=False, max_features=max_features)
    term_doc_matrix_train = vectorizer_train.fit_transform(df_train[col].values.astype('str'))

    vectorizer_test_valid = CountVectorizer(tokenizer=splitCamelCase, lowercase=False, vocabulary=vectorizer_train.vocabulary_)
    term_doc_matrix_test = vectorizer_test_valid.fit_transform(df_test[col].values.astype('str'))
    term_doc_matrix_valid = vectorizer_test_valid.fit_transform(df_valid[col].values.astype('str'))

    df_train_bow = pd.DataFrame(term_doc_matrix_train.toarray(), columns=vectorizer_train.get_feature_names_out())
    df_test_bow = pd.DataFrame(term_doc_matrix_test.toarray(), columns=vectorizer_test_valid.get_feature_names_out())
    df_valid_bow = pd.DataFrame(term_doc_matrix_valid.toarray(), columns=vectorizer_test_valid.get_feature_names_out())

    df_train_bow = df_train_bow.add_prefix(pref)
    df_test_bow = df_test_bow.add_prefix(pref)
    df_valid_bow = df_valid_bow.add_prefix(pref)

    df_train = pd.concat([df_train, df_train_bow], axis=1)
    df_test = pd.concat([df_test, df_test_bow], axis=1)
    df_valid = pd.concat([df_valid, df_valid_bow], axis=1)

    if replace_columns:
        df_train = df_train.drop(columns=[col])
        df_test = df_test.drop(columns=[col])
        df_valid = df_valid.drop(columns=[col])

    return df_train, df_test, df_valid

In [32]:
df_train, df_test, df_valid = df_train_raw, df_test_raw, df_valid_raw
print(f'train0 ########## ------> :\n {df_train.loc[[10]]}')
print(f'test0  ########## ------> :\n {df_test.loc[[10]]}')
print(f'valid0  ########## ------> :\n {df_valid.loc[[10]]}')
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'name', pref='name_', max_features=100, replace_columns=False)
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'features.enclosingScopeName', pref='enclosingScopeName_')
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'features.enclosingClassName', pref='enclosingClassName_')
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'features.variablesTypesInScope', pref='variablesTypesInScope_')
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'features.referenceType', pref='referenceType_')
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'features.scopeKind', pref='scopeKind_')
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'features.insideStatement_0', pref='insideStatement0_')
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'features.insideStatement_1', pref='insideStatement1_')
df_train, df_test, df_valid = str_to_OHE_feature(df_train, df_test, df_valid, 'features.insideStatement_2', pref='insideStatement2_')


#firstChar,features.referenceType,features.scopeKind,features.insideStatement_0,features.insideStatement_1,features.insideStatement_2

print(f'train1 ########## ------> :\n {df_train.loc[[10]]}')
print(f'test1  ########## ------> :\n {df_test.loc[[10]]}')
print(f'valid1  ########## ------> :\n {df_valid.loc[[10]]}')


train0 ########## ------> :
                                           name  correct firstChar  group  \
10  org.jetbrains.uast.UClassLiteralExpression        0         U      6   

   features.referenceType features.scopeKind features.enclosingScopeName  \
10        IMPLEMENTS_LIST                                                  

   features.variablesTypesInScope features.insideStatement_0  \
10                                                             

           features.enclosingClassName features.insideStatement_1  \
10  com.intellij.ide.actions.CutAction                              

   features.insideStatement_2  
10                             
test0  ########## ------> :
                                     name  correct firstChar  group  \
10  com.intellij.xdebugger.XDebugSession        0         X      0   

   features.referenceType features.scopeKind features.enclosingScopeName  \
10           EXTENDS_LIST                                                  

          

In [33]:
df_train.to_csv('train.csv')
df_train_raw.to_csv('train_raw.csv')

df_test.to_csv('test.csv')
df_test_raw.to_csv('test_raw.csv')

df_valid.to_csv('valid.csv')
df_valid_raw.to_csv('valid_raw.csv')

In [34]:
print(f'train ########## ------> : {df_train.shape}')
print(f'test ########## ------> : {df_test.shape}')
print(f'valid ########## ------> : {df_valid.shape}')

train ########## ------> : (271557, 416)
test ########## ------> : (218764, 416)
valid ########## ------> : (113854, 416)
